In [ ]:
import sys
import re
import copy
import time
import math
import argparse
import pickle
from functools import partial
from multiprocessing import Pool, cpu_count

import numpy as np
np.seterr(divide='ignore')
# np.seterr(all='raise')
from celluloid import Camera
from mpl_axes_aligner import align
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, ArtistAnimation
from scipy.signal import savgol_filter
from tqdm import tqdm
import h5py

import wf_func as wff
spe_pre = wff.read_model('spe.h5', 1)

import matplotlib
matplotlib.use('Agg')

In [ ]:
opdt = np.dtype([('TriggerNo', np.uint32), ('ChannelID', np.uint32), ('HitPosInWindow', np.float64), ('Charge', np.float64)])
with h5py.File('waveform/4.0-20-5.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Readout/Waveform'][:]
    tru = ipt['SimTriggerInfo/PEList'][:]
    gmu = ipt['SimTriggerInfo/PEList'].attrs['gmu']
    gsigma = ipt['SimTriggerInfo/PEList'].attrs['gsigma']
    t0truth = ipt['SimTruth/T'][:]
    l = len(ipt['Readout/Waveform'])
    print('{} waveforms will be computed'.format(l))
    window = int(len(ipt['Readout/Waveform'][0]['Waveform']) / wff.nshannon)
    assert window >= len(spe_pre[0]['spe']), 'Single PE too long which is {}'.format(len(spe_pre[0]['spe']))
    Mu = ipt['Readout/Waveform'].attrs['mu']
    Tau = ipt['Readout/Waveform'].attrs['tau']
    Sigma = ipt['Readout/Waveform'].attrs['sigma']
    gmu = ipt['SimTriggerInfo/PEList'].attrs['gmu']
    gsigma = ipt['SimTriggerInfo/PEList'].attrs['gsigma']

In [ ]:
i = 0
cid = ent[i]['ChannelID']
eid = ent[i]['TriggerNo']
truth = np.sort(tru[(tru['TriggerNo'] == eid) & (tru['PMTId'] == cid)], kind='stable', order=['TriggerNo', 'PMTId', 'HitPosInWindow'])
wave = ent[i]['Waveform'].astype(np.float) * spe_pre[ent[i]['ChannelID']]['epulse']

In [ ]:
ind = np.argwhere(wave > spe_pre[cid]['std'] * 5).flatten()
xmin = ((ind.min() - spe_pre[cid]['mar_l']) // 20 - 1) * 20
xmax = min(((ind.max() + spe_pre[cid]['mar_r']) // 20 + 1) * 20, xmin + 100)

In [ ]:
def savegif(waveform, spe_pre):
    waveform = savgol_filter(waveform, 11, 4)
    n = 0
    spe = np.append(np.zeros(len(spe_pre) - 2 * n - 1), np.abs(spe_pre))
    waveform = np.clip(waveform, 1e-6, np.inf)
    spe = np.clip(spe, 1e-6, np.inf)
    waveform = waveform / np.sum(spe)
    wave_deconv = waveform.copy()
    spe_mirror = spe[::-1]
    fig = plt.figure(figsize=(8, 6))
    camera = Camera(fig)
    ax = fig.add_subplot()
    ax2 = ax.twinx()
    ax2.set_ylim(top=1.0)
    ax.set_xlim(xmin, xmax)
    ax.set_xlabel('t/ns')
    ax.set_ylabel('Voltage/mV')
    ax2.set_ylabel('Charge')
    for i in tqdm(range(100)):
        relative_blur = waveform / np.convolve(wave_deconv, spe, mode='same')
        new_wave_deconv = wave_deconv * np.convolve(relative_blur, spe_mirror, mode='same')
        wave_deconv = new_wave_deconv
        pet = np.arange(0, len(waveform) - n)
        pwe = wave_deconv[n:] / wave_deconv[n:].sum() * waveform.sum()
        
        ax.plot(waveform * np.sum(spe), label='Waveform', color='b')
        ax.hlines(5, 0, window, color='k', label='WaveThres')
        ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'], color='g', label='Truth', linewidth=1.0, alpha=0.5)
        ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
        ax2.hlines(0.1, 0, window, color='k', linestyle='--', label='ChaThres')
        
        align.yaxes(ax, 0, ax2, 0)
        camera.snap()
    animation = camera.animate(interval=100)
    animation.save('lucyddm.gif')
    plt.close(fig)
    return

In [ ]:
savegif(wave, spe_pre[i]['spe'])